In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data = Dataset.load_builtin('ml-100k') # 무비렌즈 사이트에서 제공하는 과거 버전의 데이터 세트

trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\정성현/.surprise_data/ml-100k


In [6]:
algo = SVD()
algo.fit(trainset)

In [11]:
predictions = algo.test(testset)
print('prediction type: ', type(predictions), 'size: ',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type:  <class 'list'> size:  25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5558715675878005, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.7315554016591173, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.156820467286122, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6494797460681787, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.397224965133965, details={'was_impossible': False})]

In [18]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5558715675878005),
 ('882', '291', 3.7315554016591173),
 ('535', '507', 4.156820467286122)]

In [34]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.08   {'was_impossible': False}


In [35]:
pred = algo.predict('196', '302')
print(pred)

user: 196        item: 302        r_ui = None   est = 4.08   {'was_impossible': False}


In [37]:
# 테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이 평가
accuracy.rmse(predictions)

RMSE: 0.9492


0.9491827178380478

surprise 패키지는 반드시 데이터 칼럼의 순서가 '사용자아이디','아이템아이디','평점' 순으로 되어 있어야 한다.

In [38]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')

ratings.to_csv('ratings_noh.csv', index=False, header=None)

In [39]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))

data = Dataset.load_from_file('ratings_noh.csv', reader=reader)

In [41]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### 판다스 DataFrame에서 Surprise 데이터 세트를 로딩

In [46]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [44]:
reader = Reader(rating_scale=(0.5,5))

data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

In [45]:
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [47]:
ratings['rating2'] = ratings['rating']

In [48]:
ratings

,userId,movieId,rating,timestamp,rating2
0,1,1,4.0,964982703,4.0
1,1,3,4.0,964981247,4.0
2,1,6,4.0,964982224,4.0
3,1,47,5.0,964983815,5.0
4,1,50,5.0,964982931,5.0
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,4.0
100832,610,168248,5.0,1493850091,5.0
100833,610,168250,5.0,1494273047,5.0
100834,610,168252,5.0,1493846352,5.0


### 교차 검증과 하이퍼 파라미터 튜닝

In [3]:
# cross_validate()

from surprise.model_selection import cross_validate
import pandas as pd
from surprise import Reader, Dataset
from surprise import SVD

ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8694  0.8730  0.8722  0.8734  0.8766  0.8729  0.0023  
MAE (testset)     0.6698  0.6707  0.6702  0.6721  0.6735  0.6713  0.0014  
Fit time          4.02    3.97    4.10    4.11    4.26    4.09    0.10    
Test time         0.13    0.14    0.14    0.11    0.11    0.13    0.01    


{'test_rmse': array([0.86943315, 0.87299854, 0.87215567, 0.87339341, 0.87655795]),
 'test_mae': array([0.66980637, 0.67065475, 0.67017893, 0.67214394, 0.67353312]),
 'fit_time': (4.024359226226807,
  3.9681289196014404,
  4.100266695022583,
  4.114447116851807,
  4.2636213302612305),
 'test_time': (0.13331317901611328,
  0.14359259605407715,
  0.13588643074035645,
  0.11170125007629395,
  0.10575270652770996)}

In [9]:
# GridSearchCV

from sklearn.model_selection import GridSearchCV

param_grid = {'n_epoch':[20,40,60], 'n_factors':[50,100,200]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8764689251518029
{'n_epochs': 20, 'n_factors': 50}


# Surprise를 이용한 개인화 영화 추천 시스템 구축

In [16]:
# 학습 데이터와 테스트 데이터로 분리하지 않고 전체를 학습 데이터로 사용

from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))

data_folds = DatasetAutoFolds(ratings_file = 'ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [17]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [21]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [26]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [28]:
# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
movieIds

1073      41
1074     187
1075     223
1076     371
1077     627
1078     922
1079     923
1080    1037
1081    1095
1082    1198
1083    1270
1084    1674
1085    1987
1086    2011
1087    2012
1088    2023
1089    2300
1090    2877
1091    2901
1092    3173
1093    3328
1094    3735
1095    4131
1096    4558
1097    4993
1098    5218
1099    5378
1100    5445
1101    5447
1102    5451
1103    5481
1104    5507
1105    5841
1106    5843
1107    5872
1108    5890
1109    5891
1110    5893
1111    5902
1112    5952
1113    5956
1114    5962
1115    5965
1116    5988
1117    6001
1118    6044
Name: movieId, dtype: int64

In [29]:
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [30]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [32]:
pred = algo.predict('9', '42', verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [33]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [34]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
